# 实现通用框架无关函数

使用 {ref}`eagerpy:convert` 中显示的转换函数，已经可以定义简单的框架无关函数。

In [1]:
import eagerpy as ep

def norm(x):
    x = ep.astensor(x)
    result = x.square().sum().sqrt()
    return result.raw

这个函数可以用任何框架的原生张量来调用，它将返回该张量的范数，同样是该框架的原生张量。

```{rubric} 使用 PyTorch 张量调用 norm 函数
```

In [3]:
import torch
norm(torch.tensor([3., 4., 12.]))

tensor(13.)

```{rubric} 使用 TensorFlow 张量调用 norm 函数
```

In [2]:
import tensorflow as tf
norm(tf.constant([3., 4., 12.]))

<tf.Tensor: shape=(), dtype=float32, numpy=13.0>

如果用 EagerPy 张量来调用上面的 `norm` 函数，`ep.astensor` 调用将简单地返回它的输入。然而，最后一行中的 `result.raw` 调用仍然会提取底层的原生张量。

通常最好是实现泛型函数，它不仅透明地处理任何原生张量，还透明地处理 EagerPy 张量，即返回类型应该总是与输入类型匹配。这在像 Foolbox 这样允许用户使用 EagerPy 和原生张量的库中特别有用。为了实现这一点，EagerPy 提供了上述变换函数的两个衍生品：`ep.astensor_` 和 `ep.astensors_`。

与没有下划线的对应方法不同，它们返回额外的反转函数，用于恢复输入类型。如果 `astensor_` 的输入是原生张量，`restore_type` 将与 `.raw` 相同，但如果原始输入是 EagerPy 张量，restore_type 将不会调用 `.raw`。有了它，就可以编写通用的框架无关函数，对任何输入都透明地工作。

In [3]:
import eagerpy as ep

def norm(x):
    x, restore_type = ep.astensor_(x)
    result = x.square().sum().sqrt()
    return restore_type(result)

使用 `ep.astensors_` 变换和恢复多个输入：

In [5]:
import eagerpy as ep

def example(x, y, z):
    (x, y, z), restore_type = ep.astensors_(x, y, z)
    result = (x + y) * z
    return restore_type(result)

In [10]:
import tensorflow as tf

x = norm(tf.constant([3., 4., 12.]))

In [15]:
(x,y), tt = ep.astensors_(x, x+2)

In [23]:
norm(x+2).raw

<tf.Tensor: shape=(), dtype=float32, numpy=15.0>